In [7]:
import pandas
import numpy
import math
import datetime
import time
from time import gmtime, strftime
import os
# Import libraries use for visualization and analysis
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


import math
from __future__ import division

# Import library to execute remote commands for monasca-agent demo
#import spur
import matplotlib.pyplot as plt
#import seaborn as sns

import time as t
import datetime as dt


# We will use Keras library
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# And Scikit-Learn
from sklearn.preprocessing import MinMaxScaler

In [8]:
dataframe = pandas.read_csv("Data/training_data.csv", usecols=["cpu.wait_perc"], engine='python',sep=";")
dataframe = dataframe[1:100]
dataset = dataframe.values
dataset = dataset.astype('float32')
dataset;

# normalize the dataset thanks to scikit-learn
scaler = MinMaxScaler(feature_range=(2.3, 18.1))
dataset = scaler.fit_transform(dataset)
dataset;
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)
# X=t and Y=t+1
X, Y = create_dataset(dataset[1:10],1)

# reshape into X=t and Y=t+1

## The window 

look_back = 1

trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


# create and fit the LSTM network
model = Sequential()
model.add(LSTM(2, input_dim=look_back))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, nb_epoch=20, batch_size=1, verbose=2)

(66, 33)
Epoch 1/20
0s - loss: 111.5359
Epoch 2/20
0s - loss: 107.5791
Epoch 3/20
0s - loss: 103.7387
Epoch 4/20
0s - loss: 99.9782
Epoch 5/20
0s - loss: 96.7795
Epoch 6/20
0s - loss: 93.8116
Epoch 7/20
0s - loss: 91.0278
Epoch 8/20
0s - loss: 88.3889
Epoch 9/20
0s - loss: 85.8453
Epoch 10/20
0s - loss: 83.4090
Epoch 11/20
0s - loss: 81.0436
Epoch 12/20
0s - loss: 78.7635
Epoch 13/20
0s - loss: 76.5227
Epoch 14/20
0s - loss: 74.3690
Epoch 15/20
0s - loss: 72.2537
Epoch 16/20
0s - loss: 70.2179
Epoch 17/20
0s - loss: 68.2338
Epoch 18/20
0s - loss: 66.2932
Epoch 19/20
0s - loss: 64.4237
Epoch 20/20
0s - loss: 62.5970


In [6]:
# Estimate model performance
trainScore = model.evaluate(trainX, trainY, verbose=1)
trainScore = math.sqrt(trainScore)
trainScore = scaler.inverse_transform(numpy.array([[trainScore]]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = model.evaluate(testX, testY, verbose=1)
testScore = math.sqrt(testScore)
testScore = scaler.inverse_transform(numpy.array([[testScore]]))
print('Test Score: %.2f RMSE' % (testScore))

31/31 [==============================] - 0s
Test Score: 12.70 RMSE
